# Introduction

Download data from NREL End-Use Load Profiler (EULP) project.

1. Set up data directories and the list of files that needed to be downloaded.

2. Set up URLs and prefixed for ResStock and ComStock data on NREL's AWS data storage.

3. Download ResStock data:

    (1) County-level energy time series aggregates (amy2018 and tmy3)

    (2) County-level weather time series (amy2018 and tmy3)

    (3) State-level energy time series aggregates (amy2018 and tmy3)

    (4) Individual building energy time series (amy2018 and tmy3)
    
4. Download ComStock data for the same categories as ResStock.

# Packages

In [46]:
import os
import numpy as np
import pandas as pd
import downloader as dl


# Parameter setup and functions

In [47]:
# Choose the state of study, only supports NY or CA
state = 'ny'
# state = 'ca'

# Download tmy3 data or not
download_tmy = False

# Create directories to store the data
data_dir = f'../Data_{state}/'
resstock_amy2018_dir = os.path.join(data_dir, 'resstock_amy2018')
comstock_amy2018_dir = os.path.join(data_dir, 'comstock_amy2018')
resstock_tmy3_dir = os.path.join(data_dir, 'resstock_tmy3')
comstock_tmy3_dir = os.path.join(data_dir, 'comstock_tmy3')

# Download ResStock data

In [48]:
# Read resstock building characteristics data
res_building_state = pd.read_csv(os.path.join(data_dir, f'metadata_resstock_amy2018_{state}.csv'))

# Create a list of county IDs
county_id_df = res_building_state.loc[:, ['in.county', 'in.resstock_county_id']]
county_id_df = county_id_df.drop_duplicates()
county_id_list = county_id_df['in.county'].to_list()
county_id_list.sort()

# Create a dictionary for building id in each county
res_bldg_list_dict = dict()
for county_id in county_id_list:
    county_df = res_building_state.loc[res_building_state['in.county'] == county_id]
    building_id = county_df['bldg_id'].to_list()
    res_bldg_list_dict[county_id] = building_id

In [49]:
# ResStock building types
resstock_type_list = [
    'mobile_home',
    'multi-family_with_2_-_4_units',
    'multi-family_with_5plus_units',
    'single-family_attached',
    'single-family_detached'
]


## County-level time series aggregates

In [50]:
# Setup urls
url_root = 'https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2021/'
resstock_amy2018_pfx = url_root + 'resstock_amy2018_release_1/timeseries_aggregates/'
resstock_tmy3_pfx = url_root + 'resstock_tmy3_release_1/timeseries_aggregates/'
comstock_amy2018_pfx = url_root + 'comstock_amy2018_release_1/timeseries_aggregates/'
comstock_tmy3_pfx = url_root + 'comstock_tmy3_release_1/timeseries_aggregates/'


In [51]:
# Download resstock amy2018 data by county in NY
resstock_amy2018_county_url = resstock_amy2018_pfx + f'by_county/state={state.upper()}/'
resstock_amy2018_county_dir = os.path.join(resstock_amy2018_dir,'county')
resstock_amy2018_county_filename_list, _ = dl.download_ts_agg(resstock_amy2018_county_url,
                                                            county_id_list,
                                                            resstock_type_list,
                                                            resstock_amy2018_county_dir)


Directory already exists: ../Data_ny/resstock_amy2018\county
File cannot be found: g3600170-single-family_attached.csv
File cannot be found: g3600230-single-family_attached.csv
File cannot be found: g3600990-single-family_attached.csv


In [52]:
# Download resstock tmy3 data by county in state
if download_tmy:
    resstock_tmy3_county_url = resstock_tmy3_pfx + f'by_county/state={state.upper()}/'
    resstock_tmy3_county_dir = os.path.join(resstock_tmy3_dir,'county')
    resstock_tmy3_county_filename_list, _ = dl.download_ts_agg(resstock_tmy3_county_url,
                                                            county_id_list,
                                                            resstock_type_list,
                                                            resstock_tmy3_county_dir)


## County-level weather timeseries

In [53]:
weather_amy2018_url = url_root + 'resstock_amy2018_release_1/weather/amy2018/'
weather_amy2018_dir = os.path.join(resstock_amy2018_dir,'weather')
weather_amy2018_county_filename_list, _ = dl.download_weather_ts(weather_amy2018_url,
                                                            county_id_list,
                                                            '2018',
                                                            weather_amy2018_dir)


Directory already exists: ../Data_ny/resstock_amy2018\weather


In [54]:
weather_tmy3_url = url_root + 'resstock_tmy3_release_1/weather/tmy3/'
weather_tmy3_dir = os.path.join(resstock_tmy3_dir,'weather')
weather_tmy3_county_filename_list, _ = dl.download_weather_ts(weather_tmy3_url,
                                                            county_id_list,
                                                            'tmy3',
                                                            weather_tmy3_dir)

Directory already exists: ../Data_ny/resstock_tmy3\weather


## State-level time series aggregates

In [55]:
# Download resstock amy2018 data in state
resstock_amy2018_ny_url = resstock_amy2018_pfx + f'by_state/state={state.upper()}/'
resstock_amy2018_ny_dir = os.path.join(resstock_amy2018_dir, state)
resstock_amy2018_ny_filename_list, _ = dl.download_ts_agg(resstock_amy2018_ny_url,
                                                        [state],
                                                        resstock_type_list,
                                                        resstock_amy2018_ny_dir)


Directory already exists: ../Data_ny/resstock_amy2018\ny


In [56]:
# Download resstock tmy3 data in state
if download_tmy:
    resstock_tmy3_ny_url = resstock_tmy3_pfx + f'by_state/state={state.upper()}/'
    resstock_tmy3_ny_dir = os.path.join(resstock_tmy3_dir, state)
    resstock_tmy3_ny_filename_list, _ = dl.download_ts_agg(resstock_tmy3_ny_url,
                                                        [state],
                                                        resstock_type_list,
                                                        resstock_tmy3_ny_dir)


## Individual building time series

In [57]:
# Download resstock amy2018 building timeseries data by county in state
resstock_amy2018_bldg_pfx = 'resstock_amy2018_release_1/timeseries_individual_buildings/by_county/upgrade=0/'
resstock_amy2018_bldg_dir = os.path.join(resstock_amy2018_dir,'building')
resstock_amy2018_bldg_filename_list, _ =  dl.download_bldg_ts(url_root + resstock_amy2018_bldg_pfx, 
                                                        county_id_list, 
                                                        res_bldg_list_dict, 
                                                        resstock_amy2018_bldg_dir)


In [58]:
# Download resstock tmy3 building timeseries data by county in state
if download_tmy:
    resstock_tmy3_bldg_pfx = 'resstock_tmy3_release_1/timeseries_individual_buildings/by_county/upgrade=0/'
    resstock_tmy3_bldg_dir = os.path.join(resstock_tmy3_dir,'building')
    resstock_tmy3_bldg_filename_list, _ =  dl.download_bldg_ts(url_root + resstock_tmy3_bldg_pfx, 
                                                            county_id_list, 
                                                            res_bldg_list_dict, 
                                                            resstock_tmy3_bldg_dir)

# Download ComStock data

In [59]:
# Read comstock building characteristics data
com_building_state = pd.read_csv(os.path.join(data_dir, f'metadata_comstock_amy2018_{state}.csv'))

# Create a dictionary for building id in each county
com_bldg_list_dict = dict()
for county_id in county_id_list:
    county_df = com_building_state.loc[com_building_state['in.county'] == county_id]
    building_id = county_df['bldg_id'].to_list()
    com_bldg_list_dict[county_id] = building_id

In [60]:
# Comstock building types
comstock_type_list = [
    'fullservicerestaurant',
    'hospital',
    'largehotel',
    'largeoffice',
    'mediumoffice',
    'outpatient',
    'primaryschool',
    'quickservicerestaurant',
    'retailstandalone',
    'retailstripmall',
    'secondaryschool',
    'smallhotel',
    'smalloffice',
    'warehouse'
]


## County-level time series aggregates

In [61]:
# Download comstock amy2018 data by county in NY
comstock_amy2018_county_url = comstock_amy2018_pfx + f'by_county/state={state.upper()}/'
comstock_amy2018_county_dir = os.path.join(comstock_amy2018_dir,'county')
comstock_amy2018_county_filename_list, _ = dl.download_ts_agg(comstock_amy2018_county_url,
                                                            county_id_list,
                                                            comstock_type_list,
                                                            comstock_amy2018_county_dir)


Directory already exists: ../Data_ny/comstock_amy2018\county
File cannot be found: g3600010-hospital.csv
File cannot be found: g3600030-hospital.csv
File cannot be found: g3600030-largeoffice.csv
File cannot be found: g3600030-mediumoffice.csv
File cannot be found: g3600030-outpatient.csv
File cannot be found: g3600050-smallhotel.csv
File cannot be found: g3600070-hospital.csv
File cannot be found: g3600070-largeoffice.csv
File cannot be found: g3600090-hospital.csv
File cannot be found: g3600090-outpatient.csv
File cannot be found: g3600090-primaryschool.csv
File cannot be found: g3600110-hospital.csv
File cannot be found: g3600110-largehotel.csv
File cannot be found: g3600110-largeoffice.csv
File cannot be found: g3600110-outpatient.csv
File cannot be found: g3600110-secondaryschool.csv
File cannot be found: g3600130-largeoffice.csv
File cannot be found: g3600130-outpatient.csv
File cannot be found: g3600150-largehotel.csv
File cannot be found: g3600150-largeoffice.csv
File cannot be

In [62]:
# Download comstock tmy3 data by county in state
if download_tmy:
    comstock_tmy3_county_url = comstock_tmy3_pfx + f'by_county/state={state.upper()}/'
    comstock_tmy3_county_dir = os.path.join(comstock_tmy3_dir,'county')
    comstock_tmy3_county_filename_list, _ = dl.download_ts_agg(comstock_tmy3_county_url,
                                                            county_id_list,
                                                            comstock_type_list,
                                                            comstock_tmy3_county_dir)


## State-level time series aggregates

In [63]:
# Download comstock amy2018 data in state
comstock_amy2018_ny_url = comstock_amy2018_pfx + f'by_state/state={state.upper()}/'
comstock_amy2018_ny_dir = os.path.join(comstock_amy2018_dir, state)
comstock_amy2018_ny_filename_list, _ = dl.download_ts_agg(comstock_amy2018_ny_url,
                                                        [state],
                                                        comstock_type_list,
                                                        comstock_amy2018_ny_dir)


Directory already exists: ../Data_ny/comstock_amy2018\ny


In [64]:
# Download comstock tmy3 data in state
if download_tmy:
    comstock_tmy3_ny_url = comstock_tmy3_pfx + f'by_state/state={state.upper()}/'
    comstock_tmy3_ny_dir = os.path.join(comstock_tmy3_dir, state)
    comstock_tmy3_ny_filename_list, _ = dl.download_ts_agg(comstock_tmy3_ny_url,
                                                        [state],
                                                        comstock_type_list,
                                                        comstock_tmy3_ny_dir)


## Individual building time series

In [65]:
# Download resstock amy2018 building timeseries data by county in state
comstock_amy2018_bldg_pfx = 'comstock_amy2018_release_1/timeseries_individual_buildings/by_county/upgrade=0/'
comstock_amy2018_bldg_dir = os.path.join(comstock_amy2018_dir,'building')
comstock_amy2018_bldg_filename_list, _ = dl.download_bldg_ts(url_root + comstock_amy2018_bldg_pfx, 
                                                        county_id_list, 
                                                        com_bldg_list_dict, 
                                                        comstock_amy2018_bldg_dir)

In [66]:
# Download resstock tmy3 building timeseries data by county in state
if download_tmy:
    comstock_tmy3_bldg_pfx = 'comstock_tmy3_release_1/timeseries_individual_buildings/by_county/upgrade=0/'
    comstock_tmy3_bldg_dir = os.path.join(comstock_tmy3_dir,'building')
    comstock_tmy3_bldg_filename_list, _ = dl.download_bldg_ts(url_root + comstock_tmy3_bldg_pfx,
                                                            county_id_list, 
                                                            com_bldg_list_dict, 
                                                            comstock_tmy3_bldg_dir)